Python's `dataclasses` module provides a number of convenience operations to address a
common use-case for classes: The Dataclass, a class that primarily represents data.

In [26]:
import dataclasses


@dataclasses.dataclass
class Position:
    x: float
    y: float
    z: float


Unlike `typing.NamedTuple`, `dataclasses.dataclass` is a decorator that adds relevant
functions, instead of providing a base class.

Each field marked with an annotation is considered a field by default. With the no-arg
decorator, we get `__init__`, `__repr__`, `__eq__`, and a context aware adding of
`__hash__` based on `__eq__` and whether the dataclass is immutable.

In [27]:
pos = Position(1.0, 2.0, 3.0)
print(repr(pos))
print(str(pos))
print(f"A hash function of None since the position is mutable: {pos.__hash__}")
pos == Position(1.0, 2.0, 3.0)


Position(x=1.0, y=2.0, z=3.0)
Position(x=1.0, y=2.0, z=3.0)
A hash function of None since the position is mutable: None


True

We can also "freeze" the definition, and make other changes to the `dataclasses.dataclass`
decorator's actions by passing arguments.

In [28]:
@dataclasses.dataclass(frozen=True, order=True)
class ImmutablePoints:
    gold: int
    silver: int
    bronze: int


player_1 = ImmutablePoints(gold=3, silver=2, bronze=1)
player_2 = ImmutablePoints(gold=1, silver=20, bronze=3)

print(
    f"A hash function is available since `__eq__` and `frozen` are both True: {hash(player_1)}"
)
print(f"Player 1 is ranked higher than Player 2: {player_1 > player_2}")


A hash function is available since `__eq__` and `frozen` are both True: -925386691174542831
Player 1 is ranked higher than Player 2: True


This setup also supports defaults for fields (and the default carries to the `__init__`),
support for providing behavior after the default initialization via `__post_init__`, 
and a general `dataclasses.field` function to have finer control over field attributes.

In [29]:
@dataclasses.dataclass
class ComplexExample:
    # Normal, required field
    x: int
    # Normal, default field (NOTE: defaults must go last due to `__init__` behavior)
    y: int = 10
    # A default that isn't in the `__repr__`
    z: int = dataclasses.field(repr=False, default=10)
    # A default that evades the x = [] issue in Python
    coords: list[int] = dataclasses.field(default_factory=list)
    # A default that has arbitrary metadata
    w: int = dataclasses.field(default=0, metadata={"encoding": "<l"})

    def __post_init__(self):
        """Executes at the end of __init__"""
        if not self.coords:
            self.coords.append(self.x, self.y, self.z, self.w)


The last case, `w`, uses the `metadata` parameter for `dataclasses.field`. This parameter
takes a `typing.Mapping` or `None`, and the value passed in gets wrapped in `types.MappingProxyType`
(making it read-only). This mapping can be accessed, along with other field metadata, by
using the `dataclasses.fields` function on a class or instance. The function returns a 
tuple of `dataclasses.Field`.

In [30]:
# Returns a tuple, so we index at 4 (order matches declaration in class)
dataclasses.fields(ComplexExample)[4]


Field(name='w',type=<class 'int'>,default=0,default_factory=<dataclasses._MISSING_TYPE object at 0x000001ADC45A0550>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'encoding': '<l'}),kw_only=False,_field_type=_FIELD)

The contents returned can be inspected and used. We are interested primarily in the `name`
and `metadata` fields.

In [31]:
w_field = dataclasses.fields(ComplexExample)[4]
print(
    f"{w_field.name} is type {w_field.type} and provides metadata {w_field.metadata}."
)
print(f"{w_field.name} has encoding '{w_field.metadata.get('encoding')}'")


w is type <class 'int'> and provides metadata {'encoding': '<l'}.
w has encoding '<l'


Since this `metadata` is guaranteed to be there, we can utilize it to facilitate behavior
similar to `typing.Annotated` and bit/byte encoding.

In [32]:
import struct


@dataclasses.dataclass
class ByteEncodingExample:
    x: int = dataclasses.field(default=0, metadata={"encoding": ">h"})
    y: int = dataclasses.field(default=1, metadata={"encoding": ">H"})

    def encode(self):
        output = bytearray()
        for field in dataclasses.fields(self):
            value = getattr(self, field.name)
            # Get encoding, raise if not available.
            encoding = field.metadata["encoding"]
            output += struct.pack(encoding, value)
        return output


example = ByteEncodingExample(y=3)
example.encode()


bytearray(b'\x00\x00\x00\x03')